# iPhone or not iPhone?

by Maria Ismailova
GSOM, June 2019

In [1]:
# loading the libraries

import math
import matplotlib.pyplot as plt
import numpy as np
import os

from classification_models.resnet import ResNet18
from sklearn import metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

import keras
from keras import optimizers
from keras.callbacks import LearningRateScheduler
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.contrib import lite

Using TensorFlow backend.


# 1. Image preprocessing

Using ImageDataGenerator we will generate batches of tensor image data with real-time data augmentation; the data will be looped over. ImageDataGenerator will be used for all three samples: train sample, test sample and validation sample. The parameters will be as following:

- rotation_range = 60: degree range for random rotations
- width_shift_range = 0.5: number of pixels from interval
- height_shift_range = 0.5: number of pixels from interval
- shear_range = 0.3: shear angle in counter-clockwise direction in degrees
- zoom_range = 0.3: range for random zoom
- vertical_flip = True: randomly flip inputs vertically
- rescale = 1./255: multiplying the data by this value
- fill_mode = 'nearest': points outside the boundaries of the input are filled according to the given mode ('nearest': aaaaaaaa|abcd|dddddddd)

In [2]:
# transforming data with ImageDataGenerator

train_datagen = ImageDataGenerator(
        rotation_range = 60,
        width_shift_range = 0.5,
        height_shift_range = 0.5,
        shear_range = 0.3,
        zoom_range = 0.3,
        vertical_flip = True,
        rescale = 1./255,
        fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
        rotation_range = 60,
        width_shift_range = 0.5,
        height_shift_range = 0.5,
        shear_range = 0.3,
        zoom_range = 0.3,
        vertical_flip = True,
        rescale = 1./255,
        fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
        rescale = 1./255,
        fill_mode = 'nearest')

Using PyTorch's DataLoader class for accepting created generator; with following parameters:

- target_size = (224, 224): the dimensions to which all images found will be resized
- batch_size = 128: denotes the number of samples contained in each generated batch
- shuffle = False: keeping linear exploration scheme
- class_mode = binary: 1D numpy array of binary labels

In [3]:
# loading the data

train_generator = val_datagen.flow_from_directory(
        'full dataset/train',  
        target_size = (224, 224),  
        batch_size = 128,
        class_mode = 'binary')

val_generator = val_datagen.flow_from_directory(
        'full dataset/val',  
        target_size = (224, 224),  
        batch_size = 128,
        class_mode = 'binary')

test_generator = val_datagen.flow_from_directory(
        'full dataset/test',  
        target_size = (224, 224),  
        batch_size = 128,
        shuffle = False,
        class_mode = 'binary')

Found 48684 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
# putting indeces for classes

train_generator.class_indices

{'An': 0, 'Ip': 1}

# 2. ResNet running

To build a model we will use Residual Neural Network with 18 layers (ResNet18) with the following parameters:

- include_top is True: the fully-connected layer at the top of the network is included
- weights are imagent: pre-trained on ImageNet
- as include_top is True, input_shape = (224, 224, 3)

In [5]:
# installing residual network

!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-elp4u0_p
  Stored in directory: /tmp/pip-ephem-wheel-cache-m3g7sry1/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [6]:
# loading pretrained residual network

model = ResNet18(input_shape = (224, 224, 3), include_top = True, weights = 'imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# changing the last layer

model.layers.pop()
last = model.layers[-1].output
x = Dense (2, activation = "softmax")(last)
model = Model(model.input, x)

Before working with the model we should compile it with following parameters:

- loss: a function of error, in our case it is sparse categorical crossentropy
- optimizer: adam is used, as it shows the best convergence
- metrics: by which the quality of the model is considered, in our case it is accuracy

In [8]:
# preparing model for futher work

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [9]:
# deviding dataset into parts

batch_size = 128

In [10]:
# defining sample sizes

nb_samples_train = len(train_generator.filenames)
nb_samples_val = len(val_generator.filenames)
nb_samples_test = len(test_generator.filenames)

It is better to use adaptive learning rate: to reduce the learning rate as the number of training epochs increases.
Here we will use step decay: reduce the learning rate by a constant factor every few epochs.

In [11]:
# reducing learning rate

def step_decay(epoch):
    # initializing learning rate
    initial_lrate = 0.0001
    # initializing factor by which learning rate reduces after every epoch
    drop = 0.5
    # initializing number of iterations after which the learning rate should reduce
    epochs_drop = 5
    # writing a decay function
    lrate = initial_lrate*math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

We should initialize a callback, which will allow to define a function to invoke during program execution.

In [12]:
# initializing learning rate scheduler callback

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [13]:
# training ResNet

model.fit_generator(
        train_generator,
        steps_per_epoch = nb_samples_train/batch_size,
        epochs = 20,
        validation_data = val_generator,
        validation_steps = nb_samples_val/batch_size,
        callbacks = callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
381/380 [==============================] - 1385s 4s/step - loss: 0.5046 - acc: 0.7610 - val_loss: 0.4579 - val_acc: 0.7910
Epoch 2/20
381/380 [==============================] - 625s 2s/step - loss: 0.3580 - acc: 0.8313 - val_loss: 0.3230 - val_acc: 0.8420
Epoch 3/20
381/380 [==============================] - 628s 2s/step - loss: 0.3110 - acc: 0.8549 - val_loss: 0.3482 - val_acc: 0.8420
Epoch 4/20
381/380 [==============================] - 626s 2s/step - loss: 0.2819 - acc: 0.8698 - val_loss: 0.2821 - val_acc: 0.8600
Epoch 5/20
381/380 [==============================] - 630s 2s/step - loss: 0.2502 - acc: 0.8868 - val_loss: 0.2568 - val_acc: 0.8750
Epoch 6/20
381/380 [==============================] - 629s 2s/step - loss: 0.2384 - acc: 0.8929 - val_loss: 0.2514 - val_acc: 0.8940
Epoch 7/20
381/380 [==============================] - 629s 2s/step - loss: 0.2326 - acc: 0.8968 - val_loss: 0.2490 - val_acc: 0.8870
Epoch 8/20
381/380 [

In [14]:
# saving model

model.save('model2_mi.h5')

In [15]:
# saving weights

model.save_weights('weights2_mi.h5')

Now we need to convert model from keras to tflite.

In [16]:
# converting model

converter = lite.TFLiteConverter.from_keras_model_file('model2_mi.h5')
tflite_model = converter.convert()
open("converted_model_mi.tflite", "wb").write(tflite_model)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 100 variables.
INFO:tensorflow:Converted 100 variables to const ops.


46791100

# 3. Performance evaluating

In [17]:
# creating a function for loading and rescaling images

def load_image(img_path):

    img = image.load_img(img_path, target_size = (224, 224))
    img_tensor = image.img_to_array(img)                    
    img_tensor = np.expand_dims(img_tensor, axis = 0)         
    img_tensor /= 255.
    
    return img_tensor

In [18]:
# loading converted model

interpreter = tf.lite.Interpreter(model_path = "converted_model_mi.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [19]:
# making predictions

pred = []
for subdir, dirs, files in os.walk('full dataset/test'):
    for file in files:
        input_data = load_image(os.path.join(subdir, file))
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        pred.append(output_data[0])

In [20]:
# calculating metrics

average_precision = average_precision_score(test_generator.classes, [x[1] for x in pred])
print('Average precision is: {0:0.10f}'.format(average_precision))

Average precision is: 0.9934829185
